In [1]:
%config Completer.use_jedi = False
import sys
import warnings
warnings.filterwarnings('ignore')

import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
import numpy as np
from scipy import ndimage
import matplotlib.pyplot as plt

MEMORY = 32*1024

gpus = tf.config.list_physical_devices('GPU')
try:
    tf.config.set_logical_device_configuration(gpus[0],
                                              [tf.config.LogicalDeviceConfiguration(memory_limit=MEMORY)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
except RuntimeError as e:        
    # Memory growth must be set before GPUs have been initialized
    print(e) 
    
import pathlib
import pickle
from pathlib import Path
sys.path.append(f"{Path.home()}/morpho_repo")
sys.path.append(f"{Path.home()}/morpho_repo/turing_codebase")
from turing.utils import *
from turing.tf_utils import *
import turing.pinns as tu
from turing.loss_functions import *

import pandas as pd

1 Physical GPUs, 1 Logical GPUs


In [2]:
from pde_solvers.cn import *
from local_utils import *
from turing.three_nodes_circuits import create_circuit_3954

# Load the parameters

In [3]:
df = pd.read_csv("../with_diffusions_second_search/df_network_analysis_full_topology_with_estimates.csv")
df["adj_tup"] = df["adj_tup"].apply(lambda x: eval(f"tuple({x})"))
df["Adj"] = df["adj_tup"].apply(lambda x: np.array(x).reshape((3,3)))

In [4]:
adj=np.array([[1, 1, -1], [-1, 0, -1], [0, -1, 1]])
subnet_list = [g[1] for g in df.groupby("adj_tup") if g[0] == tuple(adj.flatten())]
if len(subnet_list) == 0:
    print("================================")
    print("There is no adjacancy matrix as: ", adj)
    print("================================")
else:
    subnet_df = subnet_list[0]

In [5]:
subnet_df.head(1)

,Unnamed: 0,min_A,min_B,min_C,avg_A,avg_B,avg_C,max_A,max_B,max_C,...,b_C_est_ratio,V_C_est_ratio,K_AC_est_ratio,K_BC_est_ratio,K_CC_est_ratio,bad_estimates,list_of_bad_estimates,loss,loss_hat,loss_ratio
0,0,0.879311,27.4671,2.00005,3.859942,31.707845,2.000238,6.759598,35.970982,2.000543,...,0.999996,0.020842,1.013337,2.379039,0.894056,2,"('V_C', 'K_BC')",9.617466e-08,2.276804e-08,0.763264


In [6]:
def load_dataset(path):
    with open(f"../{path}", "rb") as f:
        k_max, params, res = np.load(f, allow_pickle=True)
    (n_val, 
     b_A_val, mu_A_val, V_A_val, K_AA_val, K_AB_val, K_AC_val,
     b_B_val, mu_B_val, V_B_val, K_BA_val, K_BC_val,
     b_C_val, mu_C_val, V_C_val, K_CB_val, K_CC_val) = params
    params = {
              'D_A':0.01,
              'D_B':1.0,
              'n':n_val, 
              'b_A':b_A_val, 
              'mu_A':mu_A_val, 
              'V_A':V_A_val,
              'K_AA':K_AA_val, 
              'K_AB':K_AB_val,  
              'K_AC':K_AC_val,
              'b_B':b_B_val, 
              'mu_B':mu_B_val, 
              'V_B':V_B_val,
              'K_BA':K_BA_val, 
              'K_BC':K_BC_val,  
              'b_C':b_C_val, 
              'mu_C':mu_C_val, 
              'V_C':V_C_val,
              'K_CB':K_CB_val, 
              'K_CC':K_CC_val
             }
           
    return (params, res, k_max)

In [7]:
from scipy.optimize import minimize

def cos_dist(arr1, arr2):
    arr1_L = np.sqrt(np.dot(arr1, arr1))
    arr2_L = np.sqrt(np.dot(arr2, arr2))
    return np.dot(arr1, arr2)/(arr1_L*arr2_L)

def alienor_components2(epsilon, l1, bounds):
    """
    
    Args:
          epsilon (float): The accuracy of the estimates.
          l1 (float): Lipschitz constant
          bounds (list of tuples):
          bounds like [a_i, b_i], for each variables separatly.
          
          
    """
    n = len(bounds) 
    assert n >= 2, "The method expects two or more vairbales."
    assert np.all([len(item) == 2 
                   for item in bounds]), "bounds must be tuple of (a_i,b_i)"
    
    alpha = epsilon/(2*l1*np.sqrt(n-1))
    alphas = np.ones(n)
    
    def get_h_i(a,b,alpha):
        def h_i(t):
            return (a-b)*np.cos(alpha*t)/2 + (a+b)/2
        return h_i
    a,b = bounds[0]
    h_i = get_h_i(a,b,1)
    h_list = [h_i]
    for i in range(1, n):
        # [a_i, b_i]
        a,b = bounds[i]
        # alphas[i-1] (alpha/pi) / (|b_i| + |a_i|)
        alphas[i] = alpha*alphas[i-1]/(np.pi*(np.abs(b)+np.abs(a)))
        # h_i = (a_i - b_i)cos(alpha_i theta)/2 +  (a_i + b_i)/2         
        h_i = h_i = get_h_i(a,b,alphas[i])#
        h_list.append(h_i)        
        
        
    # l2 or Lipschitz constant of the aliemor h functions
    l2 = np.linalg.norm([(np.abs(d[1])+np.abs(d[0]))**2 * a**2 for d,a in zip(bounds,alphas) ])/2
    #
    theta_max = np.pi/alphas[-1]    
    return alpha, alphas, l2, h_list, theta_max

def minim_2(epsilon, l1, bounds, func, maxiter=10000):
    alpha, alphas, l2, h_list, theta_max = alienor_components2(epsilon, l1, bounds)
    k = 1
    L = l1 * l2
    theta = epsilon / L    
    theta_epsilon = theta
    
    f = lambda t: func(*[ h(t) for h in h_list])
    f_epsilon = f_theta = f(theta_epsilon)
    
    while k < maxiter:
        if theta > np.pi/alphas[-1]:            
            return k, theta,theta_epsilon, f_epsilon, ""
        
        theta = theta + (epsilon + f_theta - f_epsilon)/ L        
        f_theta  =  f(theta)
        if f_theta < f_epsilon:
            f_epsilon = f_theta
            theta_epsilon = theta
        k += 1    
    return k, theta,theta_epsilon, f_epsilon, f"max iteration '{maxiter}' is reached"

In [8]:
from multiprocessing import Pool, shared_memory

In [9]:
from multiprocessing import Pool, shared_memory

@tf.function
def grads(pinn, H):
    def flatten(arr):
        return tf.reshape(arr, (arr.shape[0]*arr.shape[1]*arr.shape[2], arr.shape[3]))
    
    with tf.GradientTape(persistent=True) as tape:        
        H = flatten(H)
        tape.watch(H)
        
        outputs = pinn.net(H)
        Ag = tf.squeeze(outputs[:, 0])
        Bg = tf.squeeze(outputs[:, 1])
        Cg = tf.squeeze(outputs[:, 2])
        

        grad_A = tape.gradient(Ag, H)
        A_x = grad_A[:, 0]
        A_y = grad_A[:, 1]
        A_t = grad_A[:, 2]

        grad_B = tape.gradient(Bg, H)
        B_x = grad_B[:, 0]
        B_y = grad_B[:, 1]
        B_t = grad_B[:, 2]


        grad_A_x = tape.gradient(A_x, H)
        A_xx = grad_A_x[:, 0]
        grad_A_y = tape.gradient(A_y, H)
        A_yy = grad_A_y[:, 1]
        grad_B_x = tape.gradient(B_x, H)
        B_xx = grad_B_x[:, 0]
        grad_B_y = tape.gradient(B_y, H)
        B_yy = grad_B_y[:, 1]
        
        
    return (tf.squeeze(Ag), tf.squeeze(A_xx), tf.squeeze(A_yy), tf.squeeze(A_t),
            tf.squeeze(Bg), tf.squeeze(B_xx), tf.squeeze(B_yy), tf.squeeze(B_t),
            tf.squeeze(Cg)
           )                               

class _LocalFunctions:
    @classmethod
    def add_functions(cls, *args):
        for function in args:
            setattr(cls, function.__name__, function)
            function.__qualname__ = cls.__qualname__ + '.' + function.__name__

In [11]:
pool_num = 15

In [12]:
bounds_A = [(0,10), (0,30), (0,200), (0,30), (0,30)]
bounds_B = [(0,10), (0,30), (0,200), (0,30), (0,30)]
bounds_C = [(0,30), (0,200), (0,30), (0,30), (0,30)]
epsilon = 2e-4#1e-3#1e-5
l1 = 1

alpha_A, alphas_A, l2_A, h_list2_A, theta_max2_A = alienor_components2(epsilon, l1, bounds_A)
alpha_B, alphas_B, l2_B, h_list2_B, theta_max2_B = alienor_components2(epsilon, l1, bounds_B)
alpha_C, alphas_C, l2_C, h_list2_C, theta_max2_C = alienor_components2(epsilon, l1, bounds_C)
L_A = l1*l2_A
L_B = l1*l2_B
L_C = l1*l2_C
print("delta theta A:", epsilon/L_A)
print("delta theta A:", L_A/epsilon)
print()
print("delta theta B:", epsilon/L_B)
print("delta theta B:", L_B/epsilon)
print()
print("delta theta C:", epsilon/L_C)
print("delta theta C:", L_C/epsilon)

batch_size = 10000
print("A epochs:", L_A/epsilon/batch_size)
print("B epochs:", L_B/epsilon/batch_size)
print("C epochs:", L_C/epsilon/batch_size)

thetas_A = np.linspace(0, theta_max2_A,  int(L_A/epsilon))
thetas_B = np.linspace(0, theta_max2_B,  int(L_B/epsilon))
thetas_C = np.linspace(0, theta_max2_C,  int(L_C/epsilon))

params_by_theta_A = np.stack([h_list2_A[0](thetas_A), h_list2_A[1](thetas_A),
                              h_list2_A[2](thetas_A), h_list2_A[3](thetas_A),
                              h_list2_A[4](thetas_A)]).T

params_by_theta_B = np.stack([h_list2_B[0](thetas_B), h_list2_B[1](thetas_B),
                              h_list2_B[2](thetas_B), h_list2_B[3](thetas_B),
                              h_list2_B[4](thetas_B)]).T

params_by_theta_C = np.stack([h_list2_C[0](thetas_C), h_list2_C[1](thetas_C),
                              h_list2_C[2](thetas_C), h_list2_C[3](thetas_C),
                              h_list2_C[4](thetas_C)]).T

theta_A_n = params_by_theta_A.shape[0]
theta_A_m = params_by_theta_A.shape[1]
theta_B_n = params_by_theta_B.shape[0]
theta_B_m = params_by_theta_B.shape[1]
theta_C_n = params_by_theta_C.shape[0]
theta_C_m = params_by_theta_C.shape[1]

delta theta A: 4e-06
delta theta A: 250000.0

delta theta B: 4e-06
delta theta B: 250000.0

delta theta C: 4.444444444444445e-07
delta theta C: 2250000.0
A epochs: 25.0
B epochs: 25.0
C epochs: 225.0


In [14]:
def to(arr):
    return arr.reshape(128, 128) 

def reshape(arr, steps=1):
    T = arr.shape[0]
    ret = np.array([
        [to(arr[i, 0, :]), to(arr[i, 1, :]), to(arr[i, 2, :])]
        for i in range(T-steps, T)
    ])
    return np.einsum("tcxy -> cxyt", ret)

def rmse(arr1, arr2):
    return np.sqrt(np.mean((arr1-arr2)**2))


def G(sigma, x, y):
    gaussian = (1/(2*np.pi*sigma**2))*np.exp(-(x**2+y**2)/(2*sigma**2))
    return gaussian

def G_discrete(sigma, n):
    l = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            l[i,j] = G(sigma, (i-(n-1)/2),(j-(n-1)/2))
    return l

def G_discrete_normalise(sigma, n):
    l = G_discrete(sigma, n)
    return l/np.sum(l)

def search_for_params(index, 
                      pinn,
                      A_max, A_min,
                      B_max, B_min,
                      C_max, C_min,
                      n_max = 20,
                      slice_min = 45, 
                      slice_max = 100,):
    print("="*40)
    path = subnet_df["path"].iloc[index]
    print(index, path)
    (params, res_1, k_max) = load_dataset(path)

    n_val = params["n"]
    mu_A_val, mu_B_val, mu_C_val = params["mu_A"], params["mu_B"], params["mu_C"]
    

    def Create_f_A(A, B, C, n, mu_A, A_diffu=None):
        def act(x, km, n=4):
            return 1 / (1 + (km / (x + 1e-20)) ** (n))

        def inh(x, km, n=4):
            return 1 / (1 + (x / (km + 1e-20)) ** (n))
        A_flat = A#.flatten()[np.newaxis, :]
        B_flat = B#.flatten()[np.newaxis, :]           
        #A_diffu =  0.01* diffu_2D(A).flatten()[np.newaxis, :]
        if A_diffu is None:
            dxdy = (10*10)/((A.shape[0]-1)*(A.shape[1]-1))
            A_diffu = ((1.0/dxdy)*(diffusion((A.shape[0],A.shape[1]),A))).flatten()[np.newaxis, :]

        def L_2_f_a(args):
            (D_A, b_A, V_A,  K_AA, K_BA
            ) = (args[:, 0:1], args[:, 1:2], args[:, 2:3], args[:, 3:4], args[:, 4:5])
            f1 = (b_A + V_A*act(A_flat, K_AA, n)*inh(B_flat, K_BA, n) - mu_A*A_flat 
                  + D_A*A_diffu)
            f2 = (b_A/(D_A+1e-6) + V_A*act(A_flat, K_AA, n)*inh(B_flat, K_BA, n)/(D_A+1e-6) 
                  - mu_A*A_flat/(D_A+1e-6) + A_diffu)

            return np.sum((f1**2 + f2**2 )/A_flat.size, axis=1)
        return L_2_f_a

    def Create_f_A_1_D(A, B, C, n, mu_A, A_diffu=None):
        def act(x, km, n=4):
            return 1 / (1 + (km / (x + 1e-20)) ** (n))

        def inh(x, km, n=4):
            return 1 / (1 + (x / (km + 1e-20)) ** (n))
        A_flat = A#.flatten()[np.newaxis, :]
        B_flat = B#.flatten()[np.newaxis, :]    
        #A_diffu =  0.01* diffu_2D(A).flatten()[np.newaxis, :]
        if A_diffu is None:
            dxdy = (10*10)/((A.shape[0]-1)*(A.shape[1]-1))
            A_diffu = ((1.0/dxdy)*(diffusion((A.shape[0],A.shape[1]),A))).flatten()[np.newaxis, :]
        def L_2_f_a(args):        
            (D_A, b_A,V_A, K_AA, K_BA
            ) = (args[0], args[1], args[2], args[3], args[4])
            f1 = (b_A + V_A*act(A_flat, K_AA, n)*inh(B_flat, K_BA, n) - mu_A*A_flat 
                  + D_A*A_diffu)
            f2 = (b_A/(D_A+1e-6) + V_A*act(A_flat, K_AA, n)*inh(B_flat, K_BA, n)/(D_A+1e-6) 
                  - mu_A*A_flat/(D_A+1e-6) + A_diffu)

            return np.sum((f1**2 + f2**2 )/A_flat.size, axis=1)
        return L_2_f_a

    def Create_f_B(A, B, C, n, mu_B, B_diffu=None):
        def act(x, km, n=4):
            return 1 / (1 + (km / (x + 1e-20)) ** (n))

        def inh(x, km, n=4):
            return 1 / (1 + (x / (km + 1e-20)) ** (n))
        A_flat = A#.flatten()[np.newaxis, :]
        B_flat = B#.flatten()[np.newaxis, :]   
        C_flat = C#.flatten()[np.newaxis, :]
        #B_diffu =  1.0* diffu_2D(B).flatten()[np.newaxis, :]
        if B_diffu is None:
            dxdy = (10*10)/((B.shape[0]-1)*(B.shape[1]-1))
            B_diffu = ((1.0/dxdy)*(diffusion((B.shape[0],B.shape[1]),B))).flatten()[np.newaxis, :]    
        def L_2_f_b(args):
            (D_B, b_B, V_B, K_AB, K_CB
            ) = (args[:, 0:1], args[:, 1:2], args[:, 2:3], args[:, 3:4], args[:, 4:5])
            f1 = (b_B + V_B*act(A_flat, K_AB, n)*inh(C_flat, K_CB, n) - mu_B*B_flat 
                 + D_B*B_diffu)
            f2 = (b_B/(D_B + 1e-6) + V_B*act(A_flat, K_AB, n)*inh(C_flat, K_CB, n)/(D_B + 1e-6)
                  - mu_B*B_flat/(D_B + 1e-6)  + B_diffu)
            return np.sum((f1**2 + f2**2 )/B_flat.size, axis=1)
        return L_2_f_b

    def Create_f_B_1_D(A, B, C, n, mu_B, B_diffu=None):
        def act(x, km, n=4):
            return 1 / (1 + (km / (x + 1e-20)) ** (n))

        def inh(x, km, n=4):
            return 1 / (1 + (x / (km + 1e-20)) ** (n))
        A_flat = A#.flatten()[np.newaxis, :]
        B_flat = B#.flatten()[np.newaxis, :]   
        C_flat = C#.flatten()[np.newaxis, :]
        #B_diffu =  1.0* diffu_2D(B).flatten()[np.newaxis, :]
        if B_diffu is None:
            dxdy = (10*10)/((B.shape[0]-1)*(B.shape[1]-1))
            B_diffu = ((1.0/dxdy)*(diffusion((B.shape[0],B.shape[1]),B))).flatten()[np.newaxis, :]
        def L_2_f_b(args):        
            (D_B, b_B, V_B,K_AB, K_CB
            )  = (args[0], args[1], args[2], args[3], args[4])
            f1 = (b_B + V_B*act(A_flat, K_AB, n)*inh(C_flat, K_CB, n) - mu_B*B_flat 
                 + D_B*B_diffu)
            f2 = (b_B/(D_B + 1e-6) + V_B*act(A_flat, K_AB, n)*inh(C_flat, K_CB, n)/(D_B + 1e-6)
                  - mu_B*B_flat/(D_B + 1e-6)  + B_diffu)
            return np.sum((f1**2 + f2**2 )/B_flat.size, axis=1)
        return L_2_f_b

    def Create_f_C(A, B, C, n, mu_C):
        def act(x, km, n=4):
            return 1 / (1 + (km / (x + 1e-20)) ** (n))

        def inh(x, km, n=4):
            return 1 / (1 + (x / (km + 1e-20)) ** (n))
        A_flat = A#.flatten()[np.newaxis, :]
        B_flat = B#.flatten()[np.newaxis, :]   
        C_flat = C#.flatten()[np.newaxis, :]
        def L_2_f_c(args):
            b_C, V_C, K_AC, K_BC, K_CC = args[:, 0:1], args[:, 1:2], args[:, 2:3], args[:, 3:4], args[:, 4:5]
            #b_C, V_C, K_AC, K_BC, K_CC = args[0], args[1], args[2], args[3], args[4]
            f = b_C + V_C*inh(A_flat, K_AC, n)*inh(B_flat, K_BC, n)*act(C_flat, K_CC, n) - mu_C * C_flat
            return np.sum(f**2, axis=1)
        return L_2_f_c

    def Create_f_C_1_D(A, B, C, n, mu_C):
        def act(x, km, n=4):
            return 1 / (1 + (km / (x + 1e-20)) ** (n))

        def inh(x, km, n=4):
            return 1 / (1 + (x / (km + 1e-20)) ** (n))
        A_flat = A#.flatten()[np.newaxis, :]
        B_flat = B#.flatten()[np.newaxis, :]   
        C_flat = C#.flatten()[np.newaxis, :]
        def L_2_f_c(args):        
            b_C, V_C, K_AC, K_BC, K_CC = args[0], args[1], args[2], args[3], args[4]
            f = b_C + V_C*inh(A_flat, K_AC, n)*inh(B_flat, K_BC, n)*act(C_flat, K_CC, n) - mu_C * C_flat
            return np.sum(f**2)
        return L_2_f_c
    ####################################################
    def flatten(arr):
        return tf.reshape(arr, (arr.shape[0]*arr.shape[1]*arr.shape[2], arr.shape[3]))   
    ####################################################
    T=1    
    L=1
    data = reshape(res_1, T)
    nodes_n = data.shape[0]
    node_names = ["A", "B", "C"]
    x_size = data.shape[1]
    y_size = data.shape[2]
    ##########################################
    # Create a mesh that is the centers of the
    # original mesh
    x_size -= 1
    y_size -= 1
    dxdy = (10 - 10/127)**2/((x_size-1)*(y_size-1))
    ##########################
    N = x_size*y_size    
    t_star = np.arange(T, T+1)
    ##########################
    x_slice = slice(slice_min, slice_max, 1)
    y_slice = slice(slice_min, slice_max, 1)

    x_range = L * np.linspace(0, x_size, x_size+1)[x_slice]
    y_range = L * np.linspace(0, y_size, y_size+1)[y_slice]

    block_x = x_range.shape[0]
    block_y = y_range.shape[0]

    x = tf.constant(x_range, dtype=tf.float32)
    y = tf.constant(y_range, dtype=tf.float32)
    # The order of the Y and X must be reversed,
    # since the chnages the value finds the derivatives
    #Y, X = tf.meshgrid(x, y)

    X, Y = tf.meshgrid(x, y)
    ts = tf.constant(t_star, dtype=tf.float32)
    T = ts[tf.newaxis, tf.newaxis, :] * tf.ones(X.shape)[:, :, tf.newaxis]
    def H_cube(X, Y, T):
        return tf.concat(
                [
                    tf.concat(
                        [
                            X[tf.newaxis, :, :, tf.newaxis],
                            Y[tf.newaxis, :, :, tf.newaxis],
                            T[:, :, i : i + 1][tf.newaxis, :, :, :],
                        ],
                        axis=3,
                    )
                    for i in range(T.shape[-1])
                ],
                axis=0,
            )
    H = H_cube(X, Y, T) 
    ##########################################
    def to(arr):
        return arr.numpy().reshape(block_x, block_y)
    
    (A, A_xx, A_yy, A_t,
     B, B_xx, B_yy, B_t,
     C, 
    ) = grads(pinn, H)
    # Rmeomve the boundary effects due to convolutions, etc.
    sub_slice = slice(2,-2,1)
    A = to(A).copy()[sub_slice, sub_slice]
    B = to(B).copy()[sub_slice, sub_slice]
    C = to(C).copy()[sub_slice, sub_slice]
    A_diff = to((A_xx + A_yy))[sub_slice, sub_slice]
    B_diff = to((B_xx + B_yy))[sub_slice, sub_slice]
    # Transform back to the original space
    A = (((A+1)*(A_max-A_min))/2) + A_min
    B = (((B+1)*(B_max-B_min))/2) + B_min
    C = (((C+1)*(C_max-C_min))/2) + C_min
    A_diff = A_diff*(A_max-A_min)/2
    B_diff = B_diff*(B_max-B_min)/2
    # Flatten the arrays
    A = A.flatten()[np.newaxis, :]
    B = B.flatten()[np.newaxis, :]
    C = C.flatten()[np.newaxis, :]
    A_diff = A_diff.flatten()[np.newaxis, :]/dxdy
    B_diff = B_diff.flatten()[np.newaxis, :]/dxdy
    #####################################################
    f_a_loss = Create_f_A(A, B, C, 4, params["mu_A"], A_diff)
    f_a_loss_1_D = Create_f_A_1_D(A, B, C, 4, params["mu_A"], A_diff)
    f_b_loss = Create_f_B(A, B, C, 4, params["mu_B"], B_diff)
    f_b_loss_1_D = Create_f_B_1_D(A, B, C, 4, params["mu_B"], B_diff)
    f_c_loss = Create_f_C(A, B, C, 4, params["mu_C"])
    f_c_loss_1_D = Create_f_C_1_D(A, B, C, 4, params["mu_C"])
    #######################################################    
    def singA(args):
        batch_id,theta_n,theta_m= args 
        params_shm = shared_memory.SharedMemory(name="params_by_theta3")
        output_shm = shared_memory.SharedMemory(name="outputs3")
        thetas = np.ndarray((theta_n,theta_m), dtype=np.float64, buffer=params_shm.buf)
        f_thetas = np.ndarray((theta_n), dtype=np.float64, buffer=output_shm.buf)
        f_thetas[batch_id*batch_size:(batch_id+1)*batch_size] = f_a_loss(thetas[batch_id*batch_size:(batch_id+1)*batch_size, :])
        return batch_id

    def singB(args):
        batch_id,theta_n,theta_m= args 
        params_shm = shared_memory.SharedMemory(name="params_by_theta3")
        output_shm = shared_memory.SharedMemory(name="outputs3")
        thetas = np.ndarray((theta_n,theta_m), dtype=np.float64, buffer=params_shm.buf)
        f_thetas = np.ndarray((theta_n), dtype=np.float64, buffer=output_shm.buf)
        f_thetas[batch_id*batch_size:(batch_id+1)*batch_size] = f_b_loss(thetas[batch_id*batch_size:(batch_id+1)*batch_size, :])
        return batch_id

    def singC(args):
        batch_id,theta_n,theta_m= args 
        params_shm = shared_memory.SharedMemory(name="params_by_theta3")
        output_shm = shared_memory.SharedMemory(name="outputs3")
        thetas = np.ndarray((theta_n,theta_m), dtype=np.float64, buffer=params_shm.buf)
        f_thetas = np.ndarray((theta_n), dtype=np.float64, buffer=output_shm.buf)
        f_thetas[batch_id*batch_size:(batch_id+1)*batch_size] = f_c_loss(thetas[batch_id*batch_size:(batch_id+1)*batch_size, :])
        return batch_id

    _LocalFunctions.add_functions(singA, singB, singC)
    
    def run(theta_n,theta_m,params_by_theta, sing, L, epsilon, batch_size):
        shm = shared_memory.SharedMemory(name="params_by_theta3",
                                         create=True, 
                                         size=params_by_theta.nbytes)
        shared_thetas = np.ndarray((theta_n,theta_m), dtype=np.float64,
                                    buffer=shm.buf)

        shared_thetas[:,:] = params_by_theta[:,:]


        f_thetas = np.zeros(theta_n)
        shm_out = shared_memory.SharedMemory(name="outputs3",create=True, size=f_thetas.nbytes)
        shared_outputs = np.ndarray((theta_n), dtype=np.float64,
                                     buffer=shm_out.buf)

        
        args =[ (batch_id, theta_n,theta_m) for batch_id in range(int(L/epsilon/batch_size) + 1)]

        with Pool(55) as pool:
            res = pool.map(sing, args)

        f_thetas[:] = shared_outputs[:]
        shm.close()
        shm.unlink()

        shm_out.close()
        shm_out.unlink()

        return f_thetas
     
    f_thetas_A = run(theta_A_n,theta_A_m,params_by_theta_A, singA, L_A, epsilon, batch_size)
    f_thetas_B = run(theta_B_n,theta_B_m,params_by_theta_B, singB, L_B, epsilon, batch_size)
    f_thetas_C = run(theta_C_n,theta_C_m,params_by_theta_C, singC, L_C, epsilon, batch_size)
    ##########################################################
    def minimise_top_n(n, h_list, bounds, loss_1_D, f_thetas, thetas):
        shift = 0
        init_params = np.zeros((n, len(h_list)))
        final_params = np.zeros((n, len(h_list)))
        init_loss = np.zeros(n)
        final_loss = np.zeros(n)

        top_n = np.argpartition(-f_thetas, -n)[-n:]
        top_n = top_n[np.argsort(f_thetas[top_n])]

        for shift in range(n):

            theta_star = thetas[top_n][0 + shift]
            init_par = tuple([h(theta_star) for h in h_list])
            init_params[shift, :] = init_par
            init_loss[shift] = f_thetas[top_n][0 + shift]
            #bounds = ((0, 200), (0, 200), (0, 200), (0, 200), (0, 200))
            res3= minimize(loss_1_D, x0=init_par, method='L-BFGS-B', bounds=bounds)#, options={'ftol':1e-10})
            final_params[shift, :] = res3['x']
            final_loss[shift] = res3['fun']

        sorted_loss_ind = np.argsort(final_loss)
        init_params = init_params[sorted_loss_ind.tolist()]
        init_loss = init_loss[sorted_loss_ind.tolist()]
        final_params = final_params[sorted_loss_ind.tolist()]
        final_loss = final_loss[sorted_loss_ind.tolist()]

        return (final_loss, final_params, init_loss, init_params )
    
    (final_loss_A, final_params_A, 
     init_loss_A, init_params_A) = minimise_top_n(n_max, h_list2_A, 
                                                  ((0, 100), (0, 500), (0, 500), (0, 500), (0, 500)), 
                                                  f_a_loss_1_D, f_thetas_A, thetas_A)

    (final_loss_B, final_params_B, 
     init_loss_B, init_params_B) = minimise_top_n(n_max, h_list2_B, 
                                                  ((0, 100), (0, 500), (0, 500), (0, 500), (0, 500)), 
                                                  f_b_loss_1_D, f_thetas_B, thetas_B)

    (final_loss_C, final_params_C, 
     init_loss_C, init_params_C) = minimise_top_n(n_max, h_list2_C, 
                                                  ((0, 500), (0, 500), (0, 500), (0, 500), (0, 500)), 
                                                  f_c_loss_1_D, f_thetas_C, thetas_C)
    
    return (path,
            (final_loss_A, final_params_A, 
             init_loss_A, init_params_A),
            (final_loss_B, final_params_B, 
             init_loss_B, init_params_B),
            (final_loss_C, final_params_C, 
             init_loss_C, init_params_C)
           )

def single_simulation(args):
    (iter_i, 
     (simulate_from_start,
     i,
     index, 
     run, 
     path, 
     (D_A_val, b_A_val, V_A_val,  K_AA_val,  K_BA_val), 
     (D_B_val, b_B_val, V_B_val,  K_AB_val,  K_CB_val), 
     (b_C_val, V_C_val,  K_AC_val,  K_BC_val, K_CC_val))) = args
        
    N=5000
    T=100
    delta_t = T/N
    model_128_10 = RD_2D_1st_Order(Ds=[D_A_val, D_B_val, 0], 
                                   delta_t=delta_t, Lx=10, Ly=10, 
                                   Ix=128, Jy=128,
                                   boundary_condition=Neumann_Boundary_2D)

    
    (params, res_1, _) = load_dataset(path)    
    if simulate_from_start:
        A_init = res_1[0, 0, :]
        B_init = res_1[0, 1, :]
        C_init = res_1[0, 2, :]
    else:
        A_init = res_1[-1, 0, :]
        B_init = res_1[-1, 1, :]
        C_init = res_1[-1, 2, :]
    
    n_val = params["n"]
    mu_A_val = params["mu_A"]
    mu_B_val = params["mu_B"]
    mu_C_val = params["mu_C"]
        
    actual_params = np.array(list(params.values())[:2] + list(params.values())[3:])
    estimated_params = np.array(
        [D_A_val, D_B_val, b_A_val, mu_A_val, V_A_val, K_AA_val, K_AB_val,
         K_AC_val, b_B_val, mu_B_val, V_B_val, K_BA_val,
         K_BC_val, b_C_val, mu_C_val, V_C_val, K_CB_val, K_CC_val])
    euclidian_dist = np.linalg.norm(actual_params-estimated_params)
    c_dist = cos_dist(actual_params, estimated_params)
    kinetics = create_circuit_3954(n_val, 
                                   b_A_val, mu_A_val, V_A_val, K_AA_val, K_AB_val, K_AC_val,
                                   b_B_val, mu_B_val, V_B_val, K_BA_val, K_BC_val,
                                   b_C_val, mu_C_val, V_C_val, K_CB_val, K_CC_val)  



    if simulate_from_start:
        res_2 = model_128_10.integrate([A_init,B_init,C_init], kinetics, 5000-1, 100)
    else:
        res_2 = model_128_10.integrate([A_init,B_init,C_init], kinetics, 5000, 100)
        
    del model_128_10
    
    with open(f"./temp/res_{index}_{run}_{i}_{0 if simulate_from_start else 1}_high_resolution_diffusion.npy", "wb") as f:
        np.save(f, res_2)
    
    return (simulate_from_start,
            i,
            index, 
            run, 
            path,
            (n_val, 
             D_A_val, D_B_val,
             b_A_val, mu_A_val, V_A_val, K_AA_val, K_AB_val, K_AC_val,
             b_B_val, mu_B_val, V_B_val, K_BA_val, K_BC_val,
             b_C_val, mu_C_val, V_C_val, K_CB_val, K_CC_val),
             euclidian_dist,
             c_dist)
    


In [15]:
n_max = 10000
results = []
for index in [0, 1, 2, 5, 9, 10, 12, 21, 23, 29, 37, 38, 39, 40, 44, 56, 62, 69, 73, 84]:
#for index in [0, 1, 5, 10, 29, 37, 39, 40, 44, 56, 62, 69, 84]:
#for index in [2,9,12, 21, 23, 38, 73]:
    for run in ["no_noise", "1_percent_noise", "2_percent_noise", "5_percent_noise"]:
        print(f"./temp/res_{index}_{run}")
        pinn = tu.NN.restore(".", f"pinn_final_{index}_{run}")
        # Load max and mins        
        with open(f"./temp/res_{index}_{run}.npy", "rb") as f:
            (A_max, A_min, 
             B_max, B_min, 
             C_max, C_min, 
             min_validation)= np.load(f, allow_pickle=True)
            print(f"A_min={A_min:.10f}, A_max={A_max:.10f}")
            print(f"B_min={B_min:.10f}, B_max={B_max:.10f}")
            print(f"C_min={C_min:.10f}, C_max={C_max:.10f}")
            print(f"min_validation={min_validation:.10f}")

        #################
        (path,
         (final_loss_A, final_params_A, 
             init_loss_A, init_params_A),
            (final_loss_B, final_params_B, 
             init_loss_B, init_params_B),
            (final_loss_C, final_params_C, 
             init_loss_C, init_params_C)
           ) = search_for_params(index, 
                      pinn,
                      A_max, A_min,
                      B_max, B_min,
                      C_max, C_min,
                      n_max,
                      slice_min = 45, 
                      slice_max = 100,)
        results.append({
          'index':index,
          'run':run,
          'path': path,
          'A_max':A_max, 
          'A_min':A_min,
          'B_max':B_max,
          'B_min':B_min,
          'C_max':C_max,
          'C_min':C_min,
          'final_loss_A':final_loss_A,
          'final_params_A':final_params_A, 
          'init_loss_A':init_loss_A,
          'init_params_A':init_params_A,
          'final_loss_B':final_loss_B, 
          'final_params_B':final_params_B, 
          'init_loss_B':init_loss_B,
          'init_params_B':init_params_B,
          'final_loss_C':final_loss_C, 
          'final_params_C':final_params_C, 
          'init_loss_C':init_loss_C, 
          'init_params_C':init_params_C
        })
        

./temp/res_0_no_noise
A_min=1.2536166807, A_max=6.3400690906
B_min=28.1674935307, B_max=35.5214285374
C_min=2.0000581460, C_max=2.0004858235
min_validation=0.0000025441
0 outputs_second_search/solution_10_0_24.npy
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
./temp/res_0_1_percent_noise
A_min=1.2691744211, A_max=6.3376066737
B_min=28.1750442621, B_max=35.4757363218
C_min=2.0000585778, C_max=2.0004847108
min_validation=0.0001276450
0 outputs_second_search/solution_10_0_24.npy
./temp/res_0_2_percent_noise
A_min=1.2683928335, A_max=6.3522614666
B_min=28.1585525209, B_max=35.4657437053
C_min=2.0000570875, C_max=2.0004859457
min_validation=0.0001412799
0 outputs_second_search/solution_10_0_24.npy
./temp/res_0_5_percent_noise
A_min=1.2493561557, A_max=6.3962258453
B_min=28.1090772973, B_max=35.4969332522
C_min=2.0000507837, C_max=2.000489

./temp/res_12_no_noise
A_min=0.2404987008, A_max=7.4204162638
B_min=6.8952767527, B_max=10.6157212793
C_min=2.0047268408, C_max=2.1407829887
min_validation=0.0006928497
12 outputs_second_search/solution_10_10_106.npy
./temp/res_12_1_percent_noise
A_min=0.2760125962, A_max=7.3600902781
B_min=6.9577019126, B_max=10.5723264707
C_min=2.0045140692, C_max=2.1358223138
min_validation=0.0006758085
12 outputs_second_search/solution_10_10_106.npy
./temp/res_12_2_percent_noise
A_min=0.2483731234, A_max=7.3673408296
B_min=6.9593224903, B_max=10.5729304304
C_min=2.0039909590, C_max=2.1357638986
min_validation=0.0002785452
12 outputs_second_search/solution_10_10_106.npy
./temp/res_12_5_percent_noise
A_min=0.1654547050, A_max=7.3917385681
B_min=6.9597864375, B_max=10.5747423093
C_min=2.0023592419, C_max=2.1362038851
min_validation=0.0008735233
12 outputs_second_search/solution_10_10_106.npy
./temp/res_21_no_noise
A_min=0.0450001654, A_max=0.6490631991
B_min=1.6157334404, B_max=2.1248533572
C_min=0.40

./temp/res_44_no_noise
A_min=0.0904517381, A_max=9.1282162414
B_min=14.7490119213, B_max=20.8363636789
C_min=2.0051448842, C_max=2.2960615786
min_validation=0.0000365830
44 outputs_second_search/solution_10_8_140.npy
./temp/res_44_1_percent_noise
A_min=0.1036461638, A_max=9.0873889174
B_min=14.7630551225, B_max=20.8163388049
C_min=2.0048676169, C_max=2.2921263811
min_validation=0.0000363943
44 outputs_second_search/solution_10_8_140.npy
./temp/res_44_2_percent_noise
A_min=0.0963454523, A_max=9.0779412729
B_min=14.7503843965, B_max=20.8220008982
C_min=2.0042515229, C_max=2.2921232053
min_validation=0.0000625903
44 outputs_second_search/solution_10_8_140.npy
./temp/res_44_5_percent_noise
A_min=0.0086126863, A_max=9.0608843166
B_min=14.7123722183, B_max=20.8475418820
C_min=2.0010219839, C_max=2.2924222540
min_validation=0.0003611558
44 outputs_second_search/solution_10_8_140.npy
./temp/res_56_no_noise
A_min=0.0330913538, A_max=0.7920515686
B_min=8.8902010676, B_max=10.7254766697
C_min=0.2

In [21]:
with open(f"./temp/estimates_results_high_resolution_diffusion.npy", "wb") as f:
    np.save(f, results)
n_max = 20
# with open(f"./temp/estimates_results_diffusion.npy", "rb") as f:
#     results = np.load(f, allow_pickle=True)

In [20]:
len(results)

80

In [22]:
n_max = 20
simulate_from_start = True
args_list = [
    (simulate_from_start,
     i,
     dict_params["index"],
     dict_params["run"],
     dict_params["path"],
     dict_params["final_params_A"][i], 
     dict_params["final_params_B"][i], 
     dict_params["final_params_C"][i]) for dict_params in results
                                       for i in range(n_max) 
]

args_list = [ (iter_i, item)
    for iter_i, item in enumerate(args_list)]

In [23]:
with Pool(pool_num) as pool:
    results2 = pool.map(single_simulation, args_list)

In [24]:
with open(f"./temp/simulation_from_start_results_high_resolution_diffusion.npy", "wb") as f:
    np.save(f, results2)

In [25]:
simulate_from_start = False
args_list = [
    (simulate_from_start,
     i,
     dict_params["index"],
     dict_params["run"],
     dict_params["path"],
     dict_params["final_params_A"][i], 
     dict_params["final_params_B"][i], 
     dict_params["final_params_C"][i]) for dict_params in results
                                       for i in range(n_max) 
]

args_list = [ (iter_i, item)
    for iter_i, item in enumerate(args_list)]

In [26]:

with Pool(pool_num) as pool:
    results3 = pool.map(single_simulation, args_list)

In [28]:
with open(f"./temp/simulation_from_end_results_high_resolution_diffusion.npy", "wb") as f:
    np.save(f, results3)